In [1]:
from operator import add
from pyspark import SparkConf, SparkContext

import warnings 
warnings.simplefilter(action='ignore')

conf = SparkConf().setMaster('local').setAppName('category-review-average')
sc = SparkContext(conf=conf)

23/05/06 22:55:36 WARN Utils: Your hostname, Keemyoui-MacBookPro.local resolves to a loopback address: 127.0.0.1; using 192.168.35.79 instead (on interface en0)
23/05/06 22:55:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/06 22:55:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/06 22:55:37 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


# 목표: Spark SQL, DataFrame, DataSet

# 1. Spark SQL의 목적
- 스파크 프로그래밍 내부에서 관계형 처리를 하기 위해
- 스키마의 정보를 이용해 자동으로 최적화를 하기 위해
- 외부 데이터셋을 사용하기 쉽게 하기 위해

# 2. Spark SQL 소개
- 스파크 위에 구현된 하나의 패키지
- 3개의 주요 API
    - SQL
    - DataFrame
    - DataSets
- 2개의 백엔드 컴포넌트
    - catalyst: 쿼리 최적화 엔진
    - Tungsten: 시리얼라이저

### 1) DataFrame
- Saprk Core에 RDD가 있다면 Spark SQL엔 DataFrame이 있다.
- DataFrame은 테이블 데이터셋이라고 보면 된다.
- 개념적으론 RDD에 스키마가 적용된것으로 보면 된다.

### 2) SparkSession
- Spark Core에 Spark Context가 있다면 Spark SQL엔 SparkSession이 있다.
- 스파크 컨텍스트와 비슷하게 새로운 세션을 만들 수 있다.
- 컨텍스트 보다는 세션을 만들어주게 될 것이다.

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("test-app").getOrCreate()

### 3) DataFrame 만들기
1. RDD에서 스키마를 정의한 다음 변형을 하거나
2. CSV, JSON 등의 데이터를 받아오면 된다.

In [5]:
# 불러오기
import csv
import random

# 랜덤 데이터 생성
data = []
for i in range(10):
    data.append((f"item{i}", random.randint(1000, 10000)))

# CSV 파일 쓰기
with open("example.csv", mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["name", "price"])
    writer.writerows(data)

print("CSV 파일 생성 완료")

CSV 파일 생성 완료


##### 3-1) RDD로부터 DataFrame 만들기
- Schema를 자동으로 유추해서 DataFrame 만들기
- Schema를 사용자가 정의하기

In [4]:
# 패키지 불러오기
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql import Row
spark = SparkSession.builder.appName("ExampleApp").getOrCreate()

# RDD:만드는 부분
lines = sc.textFile("example.csv") #sc.textFile 함수로 불러온다. (lines라는 RDD를 만든다.)
data = lines.map(lambda x: x.split(",")) # "," 기준으로 배열로 바꿔준다.
preprocessed = data.map(lambda x: Row(name=x[0], price=int(x[1]))) #각 열을 받아서 Row 만들고 있다.

# Infer: 유추해서 만드는 부분
df = spark.createDataFrame(preprocessed) # preprocessed를 넣으면 자동으로 스키마를 유추해서 DataFrame을 만들어주게 된다.

# Specify: 사용자가 스키마를 정의하는 부분
schema = StructType([
    StructField("name", StringType(), True),
    StructField("price", IntegerType(), True) #유추가 아니라 구체적으로 남기고 싶으면 이와 같이 지정할 수 있다.
])

spark.createDataFrame(preprocessed, schema).show() # schema라는 structType을 지정


23/05/06 22:56:22 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


23/05/06 22:56:25 ERROR Executor: Exception in task 0.0 in stage 0.0 (TID 0)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/keemyohaan/opt/anaconda3/envs/airflow/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 686, in main
    process()
  File "/Users/keemyohaan/opt/anaconda3/envs/airflow/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 678, in process
    serializer.dump_stream(out_iter, outfile)
  File "/Users/keemyohaan/opt/anaconda3/envs/airflow/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 273, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/Users/keemyohaan/opt/anaconda3/envs/airflow/lib/python3.10/site-packages/pyspark/rdd.py", line 1877, in takeUpToNumLeft
    yield next(iterator)
  File "/Users/keemyohaan/opt/anaconda3/envs/airflow/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0) (192.168.35.79 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/keemyohaan/opt/anaconda3/envs/airflow/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 686, in main
    process()
  File "/Users/keemyohaan/opt/anaconda3/envs/airflow/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 678, in process
    serializer.dump_stream(out_iter, outfile)
  File "/Users/keemyohaan/opt/anaconda3/envs/airflow/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 273, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/Users/keemyohaan/opt/anaconda3/envs/airflow/l

##### 3-2) 파일로부터 DataFrame 만들기
- 파일 읽어오기

In [7]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("test-app").getOrCreate()

#JSON#
dataframe = spark.read.json('dataset/nyt2.json')
#TXT Files#
dataframe_txt = spark.read_text('text_data.tx')
#CSV FILES#
dataframe_csv = spark.read.csv('csv_data.csv')
#PARQUET FILES#
dataframe_parquet = spark.read.load('parquet_data.parquet')

23/05/06 23:19:49 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


AnalysisException: Path does not exist: file:/Users/keemyohaan/Desktop/001.Python/004. Study/009.FastCampus/003.DE/01-spark/001.Notes/FastCampus/dataset/nyt2.json

### 4) CreateOrRepaceTempView
- DataFrame을 하나의 데이터베이스 테이블처럼 사용하려면, CreateOrReplaceTempView() 함수로 temporary view를 만들어줘야한다.

In [9]:
data.createOrReplaceTempView('mobility_data')
spark.sql("SELECT pickup_datetime FROM mobility_data LIMIT 5").show()

AttributeError: 'list' object has no attribute 'createOrReplaceTempView'

### 5) Python에서 Spark SQL 사용하기
- DataFrame을 RDD로 변환해 사용할수도 있다.
- Rdd = df.rdd.map(tuple)
- 하지만 RDD를 덜 사용하는 쪽이 좋다

##### 6) DataFrame의 이점
- MLLib이나 Spark Streaming 같은 다른 스파크 모듈들과 사용하기 편하다
- 개발하기 편하다
- 최적화도 알아서 된다.